In [1]:
import pandas as pd
import numpy as np
import sys, time
sys.path.append("../") # go to parent dir
from src.data.diversity_document import DiversityDocument

from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import itertools
import pyLDAvis.gensim
from IPython.display import clear_output

df = pd.read_csv('../data/external/2017-18stats.csv')
with open('../data/external/download_whitelist.txt', 'r') as f:
    whitelist = [line.strip() for line in f]
df['CompanyLinkToGPGInfo'] = df['CompanyLinkToGPGInfo'].astype(str)

df.describe()


,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,MaleLowerQuartile,FemaleLowerQuartile,MaleLowerMiddleQuartile,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile
count,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000
mean,14.470046,11.963943,15.259415,-13.792430,35.844000,34.328861,46.427434,53.572566,50.636047,49.363953,54.995196,45.004804,60.959228,39.040772
std,15.087346,15.687637,242.144758,1258.967565,36.398861,36.102908,24.114317,24.114317,26.063711,26.063711,26.172304,26.172304,24.316170,24.316170
min,-400.000000,-164.000000,-21092.000000,-120695.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.300000,0.900000,0.000000,0.000000,1.000000,0.400000,27.000000,35.550000,29.600000,27.500000,34.000000,21.200000,41.800000,17.050000
50%,13.500000,9.600000,22.500000,5.700000,20.500000,18.000000,44.400000,55.600000,48.500000,51.500000,54.000000,46.000000,63.000000,37.000000
75%,23.000000,21.100000,53.000000,38.300000,73.800000,70.900000,64.450000,73.000000,72.500000,70.400000,78.800000,66.000000,82.950000,58.200000
max,159.000000,75.700000,3848.200000,5566.700000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [2]:
#split the corpus into quartiles by the "best" companies on diversity numbers. 
upper_q = df.DiffMeanHourlyPercent.quantile(0.75)
lower_q = df.DiffMeanHourlyPercent.quantile(0.25)
df_bad = df.loc[df.DiffMeanHourlyPercent > upper_q]
df_good = df.loc[df.DiffMeanHourlyPercent < lower_q]
print('Number of companies in worst quarter: %s' % df_bad.DiffMeanHourlyPercent.count())
print('Number of companies in best quarter: %s' % df_good.DiffMeanHourlyPercent.count())

Number of companies in worst quarter: 2513
Number of companies in best quarter: 2537


In [3]:
def iter_diversity_data(df_divers):
    number_rows = df_divers.shape[0]
    counter = 0
    for index, row in df_divers.iterrows():
        counter = counter + 1
        percent_complete = (counter / number_rows) * 100
    
        if counter % 100 == 0:
            #clear_output(wait=True)
            print('%.2f percent complete' % (percent_complete))        

        doc = DiversityDocument(row)

        if doc.url_hash in whitelist:
            continue

        if not doc.has_downloaded_file and not doc.company_link == 'nan':
            doc.download()
            if doc.raw_text is None:
                with open('../data/external/download_whitelist.txt', 'a') as whitelist_file:
                    whitelist_file.write(doc.url_hash + '\n')    

        if doc.raw_text is not None and doc.clean_text is not None:
            yield doc.clean_text

    print('Processing Finished: 100%')        
    

In [4]:
class CorpusDiversity(object):
    def __init__(self, df):
        self.df = df
        self.dictionary = Dictionary(iter_diversity_data(df))

    def __iter__(self):
        for doc in iter_diversity_data(self.df):
            # tokenize each message; simply lowercase & match alphabetic chars, for now
            yield self.dictionary.doc2bow(doc)
           
#    def __len__(self):
#        return len(self.dictionary)
   
    

bad_tokenized_corpus = CorpusDiversity(df_bad)
good_tokenized_corpus = CorpusDiversity(df_good)

3.98 percent complete
7.96 percent complete
11.94 percent complete
15.92 percent complete
19.90 percent complete
23.88 percent complete
27.86 percent complete
31.83 percent complete
35.81 percent complete
39.79 percent complete
43.77 percent complete
47.75 percent complete
51.73 percent complete
55.71 percent complete
59.69 percent complete
63.67 percent complete
67.65 percent complete
71.63 percent complete
75.61 percent complete
79.59 percent complete
83.57 percent complete
87.54 percent complete
91.52 percent complete
95.50 percent complete
99.48 percent complete
Processing Finished: 100%
3.94 percent complete
7.88 percent complete
11.82 percent complete
15.77 percent complete
19.71 percent complete
23.65 percent complete
27.59 percent complete
31.53 percent complete
35.47 percent complete
39.42 percent complete
43.36 percent complete
47.30 percent complete
51.24 percent complete
55.18 percent complete
59.12 percent complete
63.07 percent complete
67.01 percent complete
70.95 percen

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [5]:
bad_corpus = bad_tokenized_corpus
good_corpus = good_tokenized_corpus

# Add bigrams and trigrams to docs (only ones that appear 5 times or more).
#bigram = Phrases(docs, min_count=5)
#for idx in range(len(docs)):
#    for token in bigram[docs[idx]]:
#        if '_' in token:
            # Token is a bigram, add to document.
#            docs[idx].append(token)

# Create a dictionary representation of the documents, and filter out frequent and rare words.
bad_dictionary = bad_tokenized_corpus.dictionary
good_dictionary = good_tokenized_corpus.dictionary

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
bad_dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
bad_dictionary.compactify()
good_dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
good_dictionary.compactify()

# This sort of "initializes" dictionary.id2token.
_ = bad_dictionary[0]
_ = good_dictionary[0]


In [6]:
print('Number of unique good tokens: %d' % len(good_dictionary))
print('Number of unique bad tokens: %d' % len(bad_dictionary))

Number of unique good tokens: 14513
Number of unique bad tokens: 10670


In [7]:
bad_lda_model = LdaModel(corpus=bad_corpus,
               id2word=bad_dictionary,
               num_topics=12, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

good_lda_model = LdaModel(corpus=good_corpus,
               id2word=good_dictionary,
               num_topics=12, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

3.98 percent complete
7.96 percent complete
11.94 percent complete
15.92 percent complete
19.90 percent complete
23.88 percent complete
27.86 percent complete
31.83 percent complete
35.81 percent complete
39.79 percent complete
43.77 percent complete
47.75 percent complete
51.73 percent complete
55.71 percent complete
59.69 percent complete
63.67 percent complete
67.65 percent complete
71.63 percent complete
75.61 percent complete
79.59 percent complete
83.57 percent complete
87.54 percent complete
91.52 percent complete
95.50 percent complete
99.48 percent complete
Processing Finished: 100%
3.98 percent complete
7.96 percent complete
11.94 percent complete
15.92 percent complete
19.90 percent complete
23.88 percent complete
27.86 percent complete
31.83 percent complete
35.81 percent complete
39.79 percent complete
43.77 percent complete
47.75 percent complete
51.73 percent complete
55.71 percent complete
59.69 percent complete
63.67 percent complete
67.65 percent complete
71.63 percen

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [8]:
# Compute Perplexity
bad_lda_corpus = [x for x in bad_corpus]
good_lda_corpus = [x for x in good_corpus]
# a measure of how good the model is. lower the better. -7.670575002821739
print('\nGood Perplexity: ', lda_model.log_perplexity(bad_lda_corpus)) 
print('\nBad Perplexity: ', lda_model.log_perplexity(good_lda_corpus)) 

3.98 percent complete
7.96 percent complete
11.94 percent complete
15.92 percent complete
19.90 percent complete
23.88 percent complete
27.86 percent complete
31.83 percent complete
35.81 percent complete
39.79 percent complete
43.77 percent complete
47.75 percent complete
51.73 percent complete
55.71 percent complete
59.69 percent complete
63.67 percent complete
67.65 percent complete
71.63 percent complete
75.61 percent complete
79.59 percent complete
83.57 percent complete
87.54 percent complete
91.52 percent complete
95.50 percent complete
99.48 percent complete
Processing Finished: 100%
3.94 percent complete
7.88 percent complete
11.82 percent complete
15.77 percent complete
19.71 percent complete
23.65 percent complete
27.59 percent complete
31.53 percent complete
35.47 percent complete
39.42 percent complete
43.36 percent complete
47.30 percent complete
51.24 percent complete
55.18 percent complete
59.12 percent complete
63.07 percent complete
67.01 percent complete
70.95 percen

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


NameError: name 'lda_model' is not defined

In [14]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(bad_lda_model, bad_lda_corpus, bad_dictionary)

/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      20.513716        1       1  0.216151 -0.191315
6      15.989564        1       2  0.170756 -0.063283
0      14.291228        1       3  0.109926 -0.315386
5      13.117273        1       4  0.120832  0.146248
8      10.215628        1       5 -0.357994 -0.065390
7       9.797081        1       6  0.174974  0.148697
11      5.397365        1       7  0.057257  0.151405
2       4.104998        1       8  0.058070  0.222419
10      2.780953        1       9 -0.173662  0.047333
3       1.886516        1      10 -0.121767  0.002654
1       1.070024        1      11 -0.167402  0.007208
9       0.835653        1      12 -0.087141 -0.090590, topic_info=      Category         Freq                  Term        Total  loglift  \
term                                                                      
174    Default  6838.000000                female  6838.000000  30.0000   
463    Default  6752.000000                 bonus  6752.000000  29.0000   
190    Default  5296.000000                  male  5296.000000  28.0000   
540    Default  4930.000000              quartile  4930.000000  27.0000   
519    Default  4199.000000                median  4199.000000  26.0000   
193    Default  4570.000000                  mean  4570.000000  25.0000   
68     Default  2717.000000               limited  2717.000000  24.0000   
191    Default  4395.000000                   man  4395.000000  23.0000   
1494   Default  1709.000000                school  1709.000000  22.0000   
2166   Default  1643.000000                   car  1643.000000  21.0000   
516    Default  2938.000000                   low  2938.000000  20.0000   
744    Default  2584.000000               finance  2584.000000  19.0000   
538    Default  2418.000000            proportion  2418.000000  18.0000   
521    Default  2552.000000                middle  2552.000000  17.0000   
1588   Default  1780.000000                    U�  1780.000000  16.0000   
181    Default  2224.000000                hourly  2224.000000  15.0000   
655    Default  2124.000000                 upper  2124.000000  14.0000   
802    Default  2616.000000                   new  2616.000000  13.0000   
215    Default  3577.000000                  role  3577.000000  12.0000   
341    Default  2148.000000                   use  2148.000000  11.0000   
1036   Default  1613.000000                 trust  1613.000000  10.0000   
331    Default  2172.000000              services  2172.000000   9.0000   
79     Default  2031.000000                  news  2031.000000   8.0000   
2644   Default  1427.000000                    K�  1427.000000   7.0000   
281    Default  3020.000000             diversity  3020.000000   6.0000   
128    Default  3576.000000                  year  3576.000000   5.0000   
144    Default  2279.000000               average  2279.000000   4.0000   
276    Default  1618.000000                cookie  1618.000000   3.0000   
123    Default  1684.000000                  view  1684.000000   2.0000   
324    Default  2647.000000               receive  2647.000000   1.0000   
...        ...          ...                   ...          ...      ...   
541    Topic12    28.813913                reciev    29.814394   4.7506   
4269   Topic12    28.699739              fakenham    29.700188   4.7504   
2940   Topic12    26.433325               mclaren    27.433704   4.7476   
4130   Topic12    26.232151                  jeep    27.232500   4.7473   
7207   Topic12    23.711740                 truro    24.712114   4.7434   
3185   Topic12    23.424788                 aston    24.425131   4.7429   
4247   Topic12    23.217737              cornwall    24.218153   4.7425   
8332   Topic12    21.385336                 škoda    22.385675   4.7390   
1551   Topic12    18.511984                 aviva    19.512768   4.7321   
10072  Topic12    16.793324  Lloyds Banking Group  

In [15]:
pyLDAvis.gensim.prepare(good_lda_model, good_lda_corpus, good_dictionary)

/Users/mrmattwright/.local/share/virtualenvs/gender-pay-gap-nlp-5378hxDS/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      26.205175        1       1 -0.214169 -0.158744
9      22.857924        1       2 -0.184845  0.112528
0      11.784695        1       3  0.368802 -0.108775
3      10.210054        1       4 -0.142191  0.068655
10      7.454581        1       5 -0.068168  0.189474
1       4.729250        1       6 -0.042009  0.079691
8       3.865192        1       7 -0.047774 -0.125274
4       3.767082        1       8 -0.092564 -0.183853
7       3.032071        1       9  0.194404  0.171589
6       2.608876        1      10  0.036259  0.180013
11      1.884634        1      11  0.157437 -0.017116
2       1.600457        1      12  0.034816 -0.208188, topic_info=      Category         Freq  \
term                          
928    Default  3444.000000   
1054   Default  4354.000000   
1071   Default  4203.000000   
1092   Default  3804.000000   
1106   Default  4349.000000   
1078   Default  3600.000000   
1079   Default  3183.000000   
370    Default  1524.000000   
1081   Default  3073.000000   
175    Default  1956.000000   
314    Default  3295.000000   
4636   Default  1111.000000   
699    Default  2412.000000   
105    Default  1316.000000   
1082   Default  1619.000000   
121    Default  1697.000000   
1105   Default  1505.000000   
308    Default  1241.000000   
970    Default  1144.000000   
3228   Default   599.000000   
537    Default  2005.000000   
918    Default  1720.000000   
181    Default   860.000000   
3795   Default   550.000000   
807    Default  2442.000000   
1004   Default   678.000000   
1464   Default   957.000000   
1074   Default  1911.000000   
596    Default   744.000000   
1454   Default   912.000000   
...        ...          ...   
14380  Topic12    43.586281   
14346  Topic12    43.586281   
14352  Topic12    38.753605   
14485  Topic12    35.600250   
14366  Topic12    33.920940   
4533   Topic12    30.932888   
5513   Topic12    29.696667   
14365  Topic12    29.088266   
14266  Topic12    28.805542   
14275  Topic12    26.220825   
3182   Topic12    25.590849   
14343  Topic12    24.255598   
14474  Topic12    23.764250   
14475  Topic12    23.764250   
14478  Topic12    23.764250   
3199   Topic12    23.440170   
105    Topic12   243.679642   
308    Topic12   221.715729   
175    Topic12   204.436432   
613    Topic12    74.172417   
1105   Topic12    97.657990   
1082   Topic12    89.539238   
101    Topic12    66.177139   
1054   Topic12   106.977913   
699    Topic12    84.832390   
64     Topic12    57.209293   
807    Topic12    74.540398   
1098   Topic12    60.011539   
1092   Topic12    65.740181   
1106   Topic12    58.040462   

                                                    Term        Total  \
term                                                                    
928                                                 care  3444.000000   
1054                                               bonus  4354.000000   
1071                                              female  4203.000000   
1092                                            quartile  3804.000000   
1106                                               woman  4349.000000   
1078                                                male  3600.000000   
1079                                                 man  3183.000000   
370                                                 time  1524.000000   
1081                                              median  3073.000000   
175                                             services  1956.000000   
314                                                 mean  3295.000000   
4636                                              arriva  1111.000000   
699                                                  low  2412.000000   
105                                              limited  1316.000000   
1082                                              middle  1619.000000   
121   